In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Burgers' equation

The inviscid Burgers' equation is the simplest nonlinear wave equation, and serves as a great stepping stone toward doing full hydrodynamics.

$$u_t + u u_x = 0$$

This looks like the linear advection equation, except the quantity being advected is the velocity itself.  This means that $u$ is no longer a constant, but can vary in space and time.

Written in conservative form, it appears as:

$$u_t + \left [ \frac{1}{2} u^2 \right ]_x = 0$$

so the flux is $F(u) = \frac{1}{2} u^2$.


Our solution method is essentially the same, aside from the Riemann problem.  We still want to use the idea of upwinding, but now we have a problem&mdash;the nonlinear nature of the Burgers' equation means that information can "pile up" and we lose track of where the flow is coming from.  This gives rise to a nonlinear wave called a shock.

For the linear advection equation, the solution was unchanged along the lines $x - ut = \mbox{constant}$&mdash;we called these the *characteristic curves*.

For Burgers' equation, the characteristic curves are given by $dx/dt = u$, but now $u$ varies in the domain.

We'll use the same finite-volume grid class

In [2]:
class FVGrid(object):

    def __init__(self, nx, ng, xmin=0.0, xmax=1.0):

        self.xmin = xmin
        self.xmax = xmax
        self.ng = ng
        self.nx = nx

        # python is zero-based.  Make easy intergers to know where the
        # real data lives
        self.ilo = ng
        self.ihi = ng+nx-1

        # physical coords -- cell-centered, left and right edges
        self.dx = (xmax - xmin)/(nx)
        self.x = xmin + (np.arange(nx+2*ng)-ng+0.5)*self.dx
        self.xl = xmin + (np.arange(nx+2*ng)-ng)*self.dx
        self.xr = xmin + (np.arange(nx+2*ng)-ng+1.0)*self.dx

        # storage for the solution
        self.a = self.scratch_array()
        self.ainit = self.scratch_array()

    def period(self, u):
        """ return the period for advection with velocity u """
        return (self.xmax - self.xmin) / np.abs(u)

    def scratch_array(self):
        """ return a scratch array dimensioned for our grid """
        return np.zeros((self.nx+2*self.ng), dtype=np.float64)

    def fill_BCs(self, atmp):
        """ fill all ghostcells with periodic boundary conditions """

        # left boundary
        for n in range(self.ng):
            atmp[self.ilo-1-n] = atmp[self.ihi-n]

        # right boundary
        for n in range(self.ng):
            atmp[self.ihi+1+n] = atmp[self.ilo+n]

    def norm(self, e):
        """ return the norm of quantity e which lives on the grid """
        if not len(e) == (2*self.ng + self.nx):
            return None

        return np.sqrt(self.dx*np.sum(e[self.ilo:self.ihi+1]**2))
    
    def plot(self):
        fig = plt.figure()
        ax = fig.add_subplot(111)

        ax.plot(self.x, self.ainit, label="initial conditions")
        ax.plot(self.x, self.a)
        ax.legend()
        return fig

The Riemann problem is more complicated.  If there is compression $u_l > u_r$, then we have a shock, and we need to compute the shock speed and then upwind based on that.  For the Burgers' equation, the shock speed is obtained via the Rankine-Hugoniot jump conditions as:

$$S = \frac{1}{2} (u_{i+1/2,L} + u_{i+1/2,R})$$

Then our Riemann problem is 

$$u_{i+1/2} = \mathcal{R}(u_{i+1/2,L}, u_{i+1/2,R}) = \left \{ \begin{array}{cc}
         u_s & \mbox{if}~ u_{i+1/2,L} > u_{i+1/2,R} \\
         u_r & \mbox{otherwise} \end{array} \right .
$$

where $u_s$ is the shock case:

$$u_s = \left \{ \begin{array}{cc}
     u_{i+1/2,L} & \mbox{if}~ S > 0 \\
     u_{i+1/2,R} & \mbox{if}~ S < 0 \end{array} \right .
$$

and $u_r$ is the rarefaction case:

$$u_r = \left \{ \begin{array}{cc}
     u_{i+1/2,L} & \mbox{if}~ u_{i+1/2,L} > 0 \\
     u_{i+1/2,R} & \mbox{if}~ u_{i+1/2,R} < 0 \\
     0           & \mbox{otherwise} \end{array} \right .
$$



<div class="alert alert-block alert-info">

**Exercise:**
    
Now let's see if we are really second-order accurate.  Compute the norm of the error after 1 period with and without the limiters for the smooth Gaussian problem.  Does it converge 2nd order?
    
</div>